# In this notebook I will:
# - read MLD climatology for chosen region
# - interpolate depth profiles for WOA temp data
# - average all temp data above MLD: GET SST FORCING

go!

## 1. of all: read MLD climatology

In [155]:
import numpy as np
import pandas as pd
import xarray as xr
import glob

In [5]:
mld1 = xr.open_dataset('MLDClimatology_DeBoyerMontagut/mld_DT02_c1m_reg2.0.nc')

In [6]:
mld1

<xarray.Dataset>
Dimensions:       (lat: 90, lon: 180, time: 12)
Coordinates:
  * lat           (lat) float32 -88.0 -86.0 -84.0 -82.0 ... 84.0 86.0 88.0 89.5
  * lon           (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * time          (time) object 0001-01-15 12:00:00 ... 0001-12-15 12:00:00
Data variables:
    krig_std_dev  (time, lat, lon) float32 ...
    mask          (lat, lon) float32 ...
    med_dev       (time, lat, lon) float32 ...
    mld           (time, lat, lon) float32 ...
    mld_raw       (time, lat, lon) float32 ...
    mld_smth      (time, lat, lon) float32 ...
    n_profiles    (time, lat, lon) float32 ...
Attributes:
    File_Name:              mld_T02_ALL_clim_1941_2008.nc
    Producer_Agency:        IFREMER - LOS - Tue Nov  4 22:11:08 2008
    Methodology:            de Boyer Montegut et al., JGR 2004 (http://www.lo...
    Associate_File:         06_test_bl_thadathil_as/B_PREGRID/PREGRID_ALL_mld...
    Source_Files:           WOCE Global Data V

In [115]:
def MLD_at_latlon(lats=40,lons=180-20, mlddata=mld1, justMLD=True):
    """This function returns MLD depth across the year at a specific location"""
    #ToDO include check whether the spot is on land or water
    #Make sure the lat lon usage across everything is coherent!
    
    mld2 = mlddata.sel(lat=lats,lon=lons,method='nearest').to_dataframe()
    mld2['time'] = mld2.index
    mld2.index = range(1,13)
    mld2.index.names = ['time']
    
    if justMLD==True:
        return mld2[['mld']]
    else:
        return mld2
    
#MLD_at_latlon()
MLD4020 = MLD_at_latlon()

In [42]:
TempNCDFfiles = glob.glob('WOA2018/Temperature/*.nc', recursive=True)

WOAtemp = xr.open_mfdataset(TempNCDFfiles,combine='by_coords',decode_times=False)

In [99]:
len([i for j in range(0,57) for i in range(1,13)])
#len(WOAtemp_df.index.get_level_values(level='depth'))

684

In [150]:
def WOATempInLocation(lats=40,lons=-20, WOAtempdata=WOAtemp, justTempDepth=True):
    """This function returns the xarray of Temperature in a specific location"""
    WOAtemp1 = WOAtempdata.sel(nbounds=0)
    WOAtemp_df = WOAtemp1.sel(lat=lats,lon=lons, method='nearest').to_dataframe()
    
    WOAtemp_df.index = pd.MultiIndex.from_arrays([WOAtemp_df.index.get_level_values(level='depth'), [i for j in range(0,57) for i in range(1,13)]])
    
    WOAtemp_df.index.names = ['depth','time']
    
    if justTempDepth==True:
        return WOAtemp_df[['t_an']]
    else:
        return WOAtemp_df

    
WOAtempprofiles = WOATempInLocation()

In [152]:
print(WOAtempprofiles.head())
MLD4020.head()

                t_an
depth time          
0.0   1     14.87860
      2     14.39471
      3     14.18719
      4     14.59340
      5     15.61929


,mld
time,
1,119.166565
2,131.147095
3,106.412193
4,58.556564
5,32.214375


In [168]:
for i in range(1, 13):
    (depthindx,) = (ncdepth >= mlddat.outForcing[i]).nonzero()
    ncdat_mld[i, :, :] = ncdat[i, depthindx.min(), :, :]

NameError: name 'ncdepth' is not defined

In [178]:
for i in range(1, 12):
    print(WOAtempprofiles.loc[(1500, np.float(i) )])

t_an    5.68191
Name: (1500.0, 1), dtype: float32
t_an    5.47051
Name: (1500.0, 2), dtype: float32
t_an    5.31509
Name: (1500.0, 3), dtype: float32
t_an    5.42031
Name: (1500.0, 4), dtype: float32
t_an    5.5075
Name: (1500.0, 5), dtype: float32
t_an    5.4514
Name: (1500.0, 6), dtype: float32
t_an    5.55961
Name: (1500.0, 7), dtype: float32
t_an    5.46881
Name: (1500.0, 8), dtype: float32
t_an    5.4038
Name: (1500.0, 9), dtype: float32
t_an    5.40901
Name: (1500.0, 10), dtype: float32
t_an    5.55111
Name: (1500.0, 11), dtype: float32


In [73]:
#ds2.variables
ds3 = ds2.sel(nbounds=0, time=371.5+1)

df = ds3.sel(lat=40,lon=-20, method='nearest').to_dataframe()

df

lat   lon         crs  lat_bnds  lon_bnds  depth_bnds  \
depth  time                                                            
0.0    372.5  40.5 -19.5 -2147483647      40.0     -20.0         0.0   
       373.5  40.5 -19.5 -2147483647      40.0     -20.0         0.0   
       374.5  40.5 -19.5 -2147483647      40.0     -20.0         0.0   
       375.5  40.5 -19.5 -2147483647      40.0     -20.0         0.0   
       376.5  40.5 -19.5 -2147483647      40.0     -20.0         0.0   
...            ...   ...         ...       ...       ...         ...   
1500.0 379.5  40.5 -19.5 -2147483647      40.0     -20.0      1475.0   
       380.5  40.5 -19.5 -2147483647      40.0     -20.0      1475.0   
       381.5  40.5 -19.5 -2147483647      40.0     -20.0      1475.0   
       382.5  40.5 -19.5 -2147483647      40.0     -20.0      1475.0   
       383.5  40.5 -19.5 -2147483647      40.0     -20.0      1475.0   

              climatology_bounds      t_an       t_mn  t_dd      t_sd  \
depth  time                                                             
0.0    372.5                 0.0  14.87860  14.765243   8.0  0.403968   
       373.5                 1.0  14.39471  14.585494  23.0  0.552522   
       374.5                 2.0  14.18719  14.013444  18.0  0.814553   
       375.5                 3.0  14.59340  14.430242  13.0  0.645615   
       376.5                 4.0  15.61929  15.573319  15.0  0.987385   
...                          ...       ...        ...   ...       ...   
1500.0 379.5                 7.0   5.46881   5.303387   7.0  0.305868   
       380.5                 8.0   5.40380   5.493583  13.0  0.453310   
       381.5                 9.0   5.40901   5.163136   5.0  0.434501   
       382.5                10.0   5.55111   5.697942   5.0  0.495621   
       383.5                11.0   5.40441   5.751211   3.0  0.359141   

                  t_se      t_oa      t_ma  t_gp  
depth  time                                       
0.0    372.5  0.142824 -0.113358 -2.377490  67.0  
       373.5  0.115209  0.190784 -2.861381  67.0  
       374.5  0.191992 -0.173746 -3.068900  67.0  
       375.5  0.179061 -0.163158 -2.662690  67.0  
       376.5  0.254942 -0.045971 -1.636800  67.0  
...                ...       ...       ...   ...  
1500.0 379.5  0.115607 -0.165423 -0.001480  67.0  
       380.5  0.125726  0.089783 -0.066490  67.0  
       381.5  0.194315 -0.245874 -0.061280  66.0  
       382.5  0.221648  0.146832  0.080820  67.0  
       383.5  0.207350  0.346801 -0.065880  66.0  

[684 rows x 15 columns]